In [35]:
import numpy as np
import matplotlib.pyplot as plt

n_runs = 2 # Number of runs to average over

# Constants
k = 1.380649e-23 # Boltzmann constant (J/K)
T_star = 2 # Temperature (K)
kT = k*T_star # Product of Boltzmann constant and temperature (J)
epsilon = 1.0  # Depth of potential well
sigma = 1.0  # Finite distance where potential is zero
num_molecules = 20  # Number of molecules
max_displacement = 0.01  # Maximum displacement in a single step
num_steps = 1000  # Number of steps for the simulation
densities = np.linspace(0.1, 0.8, 10)  # Different densities to simulate
pressures = []  # Store the average pressure for each density


def lennard_jones_potential(r):
    """ Calculate Lennard-Jones potential for a distance r in reduced units """
    r6 = (1 / r)**6  # sigma is 1 in reduced units
    r12 = r6**2
    return 4 * (r12 - r6)  # epsilon is 1 in reduced units

def total_potential_energy(positions, box_size):
    # Compute all pairwise distances using broadcasting
    diff = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    diff = diff - np.round(diff / box_size) * box_size  # Periodic boundary conditions
    distances = np.sqrt(np.sum(diff**2, axis=-1))

    # Avoid division by zero and self-interactions
    np.fill_diagonal(distances, np.inf)
    
    # Get only the upper triangle of the distance matrix
    distances = distances[np.triu_indices(num_molecules, k=1)]

    # Vectorized Lennard-Jones potential calculation
    return np.sum(lennard_jones_potential(distances)) 

def metropolis_step(positions, total_energy, box_size):
    """ Perform one step of the Metropolis algorithm """
    # Choose a random molecule
    molecule_idx = np.random.randint(num_molecules)
    old_position = positions[molecule_idx].copy()

    # Move the molecule to a new position
    displacement = (np.random.rand(2) - 0.5) * max_displacement
    positions[molecule_idx] += displacement

    # Apply periodic boundary conditions
    positions[molecule_idx] %= box_size

    # Calculate the energy change
    new_energy = total_potential_energy(positions, box_size)
    delta_energy = new_energy - total_energy

    # Metropolis criterion
    if delta_energy > 0 and -delta_energy / T_star < np.log(np.random.rand()):
        # Reject the move, revert to the old position
        positions[molecule_idx] = old_position
    else:
        # Accept the move, update total energy
        total_energy = new_energy

    return positions, total_energy

def calculate_pressure(positions, box_size):
    """ Calculate pressure using the virial theorem """
    diff = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    diff = diff - np.round(diff / box_size) * box_size
    distances = np.sqrt(np.sum(diff**2, axis=-1))
    np.fill_diagonal(distances, np.inf)
    forces = lennard_jones_force(distances)
    forces = np.triu(forces, 1)
    distances = np.triu(distances, 1)
    virial = np.sum(forces*distances) / 3
    beta = 1 / kT
    density = num_molecules / box_size**2
    volume = box_size**2
    pressure = density / beta + virial / volume
    return pressure

def lennard_jones_force(distances):
    """ Calculate Lennard-Jones force for a distance array """
    return 48 * epsilon * (sigma**12 / distances**13 - 0.5 * sigma**6 / distances**7)

In [36]:
# Main simulation
np.random.seed(0)
for density in densities:
    box_size = np.sqrt(num_molecules / density)
    positions = np.random.rand(num_molecules, 2) * box_size

    # Initial total energy
    total_energy = total_potential_energy(positions, box_size)

    pressures_for_density = []
    for step in range(num_steps):
        positions, total_energy = metropolis_step(positions, total_energy, box_size)
        if step > num_steps // 2:  # Collect data after reaching equilibrium
            pressure = calculate_pressure(positions, box_size)
            pressures_for_density.append(pressure)

    avg_pressure = np.mean(pressures_for_density)
    pressures.append(avg_pressure)

In [37]:
# Plotting Density vs Pressure
plt.figure(figsize=(10, 6))
plt.plot(densities, pressures, marker='o', linestyle='-')
plt.title("Pressure vs Density")
plt.xlabel("Density")
plt.ylabel("Pressure")
plt.grid(True)
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp4etxjv7p 45b35e7c45bf1f94c12d11a2e5717b88.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./45b35e7c45bf1f94c12d11a2e5717b88.tex
LaTeX2e <2023-11-01>
L3 programming layer <2023-11-09>
(/usr/local/texlive/2023basic/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023basic/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.7 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on tmp4etxjv7p/45b35e7c45bf1f94c12d11a2e571

<Figure size 1000x600 with 1 Axes>